# Functions testing for dataGen

In [1]:
import sys, os, re
import pandas as pd
import numpy as  np

# def find(ids):

In [2]:
def find(ids):
    file_list_total = os.listdir('id_temp')

    file_list = []

    for i in range(len(file_list_total)):
        if len(file_list_total[i]) > 4:
            continue

        else:
            file_name = str(file_list_total[i])
            file_list.append(int(file_name))

    file_list.sort()

    for file in file_list:
        if file % 100 == 0:
            file_last = file
        else:
            exit

    return file_last

print(find('id_temp'))    

1100


# ShearStress

In [33]:
def shear(id):
    with open(f'{id}/OpenFOAM_Template/constant/polyMesh/boundary', 'r') as fboundary:
        temp = re.split('(aerofoil)', fboundary.read())[2]
        for item in re.finditer('(nFaces[\s]*)([\w]*)', temp):
            nFaces = int(item.group(2))
        for item in re.finditer('(startFace[\s]*)([\w]*)', temp):
            startFace = int(item.group(2))
        endFace = startFace + nFaces
    
    with open(f'{id}/OpenFOAM_Template/constant/polyMesh/faces', 'r') as ffaces:
        faces = re.findall('\(.*\)', ffaces.read())
        face_total = []; Faces = []
        for item in faces:
            face_total.append(int(re.findall('[\w]*', item)[1]))
        Faces = face_total[startFace:endFace]

    with open(f'{id}/OpenFOAM_Template/constant/polyMesh/points', 'r') as fpoints:
        points = re.findall('\(.*\)', fpoints.read())
        x_total = []; x = []
        y_total = []; y = []
        for index in range(len(points)):
            for item in re.finditer('(\()([\S ]*)(\))', points[index]):
                x_total.append(float(re.findall('[\S]*', item.group(2))[0]))
                y_total.append(float(re.findall('[\S]*', item.group(2))[2]))
        for item in Faces:
            x.append(x_total[item])
            y.append(y_total[item])
    
    with open(f'{id}/1103/wallShearStress', 'r') as fstress:
        stress = fstress.read().replace('(0 0 0)', '')
        stress = re.findall('\(.*\)', stress)
        Tau_x = []; Tau_y = []
        for item in stress:
            for item in re.finditer('(\()([\S ]*)(\))', item):
                Taudata = item.group(2)
                Tau_x.append(float(re.split('[\ ]', Taudata)[0]))
                Tau_y.append(float(re.split('[\ ]', Taudata)[1]))

    df = pd.DataFrame(x, columns={'x'})
    df['y'] = y
    df = pd.DataFrame(x, columns={'x'})
    df['y'] = y
    df['shear stress x'] = Tau_x
    df['shear stress y'] = Tau_y
    df['shear stress'] = np.sqrt(np.square(df['shear stress x']) + np.square(df['shear stress y']))
    q = 0.5 * np.square(25.75)
    df['shear stress'] = df['shear stress'] / q
    df_yp = df[df['y'] >= 0]
    df_yn = df[df['y'] <= 0]
    df_up = df_yp.sort_values('x')
    df_down = df_yn.sort_values('x')

    os.mkdir('temp')
    df_up.to_csv(f'temp/{id}_shear_upper.csv')
    df_down.to_csv(f'temp/{id}_shear_bottom.csv')

    return(0)
if shear('id_temp') != 0:
    print('\tcan not import the shear data.')

id = 'id_temp'
dfup = pd.read_csv(f'temp/{id}_shear_upper.csv')
dfdown = pd.read_csv(f'temp/{id}_shear_bottom.csv')
dfup

,Unnamed: 0,x,y,shear stress x,shear stress y,shear stress
0,72,0.000228,0.003098,-3.36343,0.390021,0.010213
1,378,0.001190,0.007990,-3.43353,-0.487070,0.010460
2,97,0.003057,0.012618,-3.71385,0.276756,0.011233
3,48,0.005743,0.016827,-3.28124,0.404606,0.009972
4,333,0.009115,0.020507,-3.87162,-0.145142,0.011686
...,...,...,...,...,...,...
228,194,0.990012,0.000301,-3.30561,-0.530491,0.010098
229,12,0.990102,0.001370,-3.20158,0.442967,0.009749
230,265,0.995008,0.000144,-3.86544,0.140216,0.011667
231,337,0.995053,0.000687,-3.85527,-0.168600,0.011640


In [32]:
os.system(f'rm -r temp')

0